<a href="https://colab.research.google.com/github/Shkev/Hotdog_notHotdog/blob/master/hotdog_notHotdog_nn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Hotdot Not Hotdog Neural Network

In [0]:
try:
  tpu = tf.distribute.cluster_resolver.TPUClusterResolver() # TPU detection
except ValueError: # If TPU not found
  tpu = None
  
tf.tpu.experimental.initialize_tpu_system(tpu)
print('Running on TPU ', tpu.cluster_spec().as_dict()['worker']) 

INFO:tensorflow:Initializing the TPU system.
INFO:tensorflow:Finished initializing TPU system.
Running on TPU  ['10.22.114.26:8470']


In [0]:
import tensorflow as tf
from tensorflow.contrib import tpu
from tensorflow.contrib.cluster_resolver import TPUClusterResolver
from tensorflow.contrib.tpu.python.tpu import tpu_function
import os
from tqdm import tqdm
from matplotlib import pyplot as plt

tpu_address = 'grpc://' + os.environ['COLAB_TPU_ADDR']
tpu_cluster = TPUClusterResolver(
    tpu=[tpu_address]).get_master()

tpu_function.get_tpu_context().set_number_of_shards(8)

#hyperparam
training_iterations = 50

PATH = "/content/drive/My Drive/Colab Notebooks/ML/Hotdog_notHotdog/trainingDataHotDog"
validation_path = "/content/drive/My Drive/Colab Notebooks/ML/Hotdog_notHotdog/validationDataHotDog"
saver_path = "/content/drive/My Drive/Colab Notebooks/ML/Hotdog_notHotdog/Saved_Model_Checkpoint/nn_model.ckpt"

##Defining All Functions


---

Label Creation:

In [0]:
def y_labels(index):
    return tf.one_hot([index], 3)     #creates one_hot encoding for data; tensor size = (# of elements in index) x (depth)

Importing Data:

In [0]:
def import_data(image_path):
  image_contents = tf.read_file(filename=image_path)   #reads image file
  modified_image = tf.image.decode_jpeg(contents=image_contents, channels=1)   #decodes jpeg image to tensor (uint8)
  cropped_and_modified_image_tensor = tf.image.resize_with_crop_or_pad(modified_image, 100, 100)   #resizes image
  image_tensor = tf.cast(tf.reshape(cropped_and_modified_image_tensor, [1, 10000]), dtype=tf.float32)   #casts reshaped modified image tensor as a float32
  return image_tensor

###**Neural Network**

In [0]:
def neural_network(input_layer):
  Weight_net_1 = {'weights': tf.Variable(tf.random_normal(shape=(10000, 32)), dtype=tf.float32, name="Weight1"), 'bias': tf.Variable(tf.random_normal(shape=(1, 1)), dtype=tf.float32, name="Bias1")}
  Weight_net_2 = {'weights': tf.Variable(tf.random_normal(shape=(32, 16)), dtype=tf.float32, name="Weight2"), 'bias': tf.Variable(tf.random_normal(shape=(1, 1)), dtype=tf.float32, name="Bias2")}
  Weight_net_3 = {'weights': tf.Variable(tf.random_normal(shape=(16, 3)), dtype=tf.float32, name="Weight3"), 'bias': tf.Variable(tf.random_normal(shape=(1, 1)), dtype=tf.float32, name="Bias3")}
  #Weight_net_4 = {'weights': tf.Variable(tf.random_normal(shape=(16, 3)), dtype=tf.float32, name="Weight4"), 'bias': tf.Variable(tf.random_normal(shape=(1, 1)), dtype=tf.float32, name="Bias4")}
  
  #Input Layer
  hypothesis = input_layer
  #Hidden Layer 1
  hypothesis = tf.nn.relu(tf.matmul(hypothesis, Weight_net_1['weights']) + Weight_net_1['bias'])
  #Hidden Layer 2
  hypothesis = tf.nn.relu(tf.matmul(hypothesis, Weight_net_2['weights']) + Weight_net_2['bias'])
  #Hidden Layer 3
  hypothesis = tf.nn.softmax(tf.matmul(hypothesis, Weight_net_3['weights']) + Weight_net_3['bias'])
  #output cell
  #hypothesis = tf.nn.softmax(tf.matmul(hypothesis, Weight_net_4['weights']) + Weight_net_4['bias'])
  return hypothesis

##Running Code

Declaring Placeholders:

In [0]:
X = tf.placeholder(shape=[1, 10000], dtype=tf.float32, name='X')
Y = tf.placeholder(shape=[1, 3], dtype=tf.float32, name='Y')

In [0]:
nn = neural_network(input_layer=X)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels=Y, logits=nn), name='cross_entropy')
train = tf.contrib.tpu.CrossShardOptimizer(
    tf.train.AdamOptimizer(1)).minimize(cross_entropy)  #minimizes cross entropy loss


#to save nn object (saver function)
#saver = tf.train.Saver(max_to_keep=1, name='saver')

#to store sigmoid determinant
sigmoid_determinant = []

**Starting Session to run code:**


In [0]:
with tf.Session(tpu_cluster) as sess:
  sess.run(tpu.initialize_system())
  sess.run(tf.global_variables_initializer())
  sess.run(tf.local_variables_initializer())
  #new_saver = tf.train.import_meta_graph(os.path.join(saver_path, 'nn_model.ckpt.meta'))
  #new_saver.restore(sess, tf.train.latest_checkpoint(saver_path))  
  for iteration in tqdm(range(training_iterations - 0), desc="COMPLETION", ncols=80):
    image_folder_dirs = [os.path.join(PATH, file_name) for file_name in os.listdir(PATH)]   #finding the number image folders
    index_label = 0                          #setting the index labels to 0
    for element in image_folder_dirs:        #the number of image_elements with in each of the image folders
      label = sess.run(y_labels(index=index_label))     #creating the Y_labels for the network
      image_tensor = sess.run(import_data(element))
      sess.run(train, feed_dict={X:image_tensor, Y:label}) #begin training
      index_label += 1     #incrementing number of labels
    #saver.save(sess, saver_path, global_step=iteration)
  
  while 1:
    validation_folder_dirs = [os.path.join(validation_path, file_name) for file_name in os.listdir(validation_path)]
    for image in validation_folder_dirs:
      validation_input = sess.run(import_data(image))
      prediction = sess.run(nn, feed_dict={X:validation_input})
      sigmoid = sess.run(tf.nn.sigmoid(prediction))
      sigmoid.append(sigmoid_determinant)
      
      plt.ylabel('Sigmoid')
      plt.xlabel('Iteration')
      plt.title('Sigmoid Determinant vs. Iterations')
      plt.tight_layout()
      plt.plot(sigmoid_determinant, label='Validation')
      
  sess.run(tpu.shutdown_system())